In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
class individual:
    
    def __init__(self, initial_range, individual_size):
        self.genes = np.random.uniform(low=-initial_range,high=initial_range,size=(individual_size))
        self.evaluated = False
        self.fitness = np.zeros((8))
        self.player_energy = np.zeros((8))
        self.enemy_energy = np.zeros((8))
        
class log_book():
    
    def __init__(self):
        self.logs = []
        
    def create_log(self,pop,gen,nevals,train_enemies,hof_gain,hof_defeat):
        log = {}
        train_gains = np.array([sum(ind.gains[train_enemies-1]) for ind in pop])
        all_gains = np.array([ind.sum_gain for ind in pop])
        all_defeated = np.array([ind.defeated for ind in pop])
        log['gen'] = gen
        log['nevals'] = nevals
        log['train_enemies_gain_mean'] = train_gains.mean()
        log['train_enemies_gain_max'] = train_gains.max()
        log['train_enemies_gain_std'] = train_gains.std()
        log['all_enemies_gain_mean'] = all_gains.mean()
        log['all_enemies_gain_max'] = all_gains.max()
        log['all_enemies_defeated_max'] = all_defeated.max()
        log['hof_gain_max'] = hof_gain['max_value']
        log['hof_defeat_max'] = hof_defeat['max_value']
        self.logs.append(log)
        self.pd = pd.DataFrame(self.logs)
        
def play(x):
    global env
    if x.evaluated:
        return x
    else:
        for enemy in range(1,9):
            env.update_parameter("enemies",[enemy])
            res = env.play(x.genes)
            x.fitness[enemy-1] = res[0]
            x.player_energy[enemy-1] = res[1]
            x.enemy_energy[enemy-1] = res[2]
        x.gains = (x.player_energy-x.enemy_energy)
        x.sum_gain = x.gains.sum()
        x.defeated = (x.enemy_energy==0).sum()
        x.evaluated = True
        return x

In [4]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5

In [5]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1,child2 = individual(0,len(parent1.genes)), individual(0,len(parent1.genes))
    child1.genes = (alpha*parent1.genes)+((1-alpha)*parent2.genes)
    child2.genes = (alpha*parent2.genes)+((1-alpha)*parent1.genes)
    return child1,child2

def cxBlend(parent1,parent2,alpha):
    child1,child2 = individual(0,len(parent1.genes)), individual(0,len(parent1.genes))
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1.genes,))) - alpha
    child1.genes = (gamma*parent1.genes)+((1-gamma)*parent2.genes)
    child2.genes = (gamma*parent2.genes)+((1-gamma)*parent1.genes)
    return child1, child2

In [6]:
def mutGaussian(ind,sigma,indpb):
    new_ind = individual(0,len(ind.genes))
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind.genes))]
    new_ind.genes = ind.genes + np.array(mutation)
    return new_ind

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_ind = individual(0,len(ind.genes))
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind.genes),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    new_ind.genes = ind.genes + np.array(mutation)
    return new_ind

def selTournament(individuals,k,tourn_size,train_enemies):
    selected = []
    for i in range(k):
        tourn_selection = np.random.choice(individuals,tourn_size)
        best = np.argmax([ind.gains[train_enemies-1].sum() for ind in tourn_selection])
        selected.append(tourn_selection[best])
    return selected

In [62]:
def best_initializer(pop_size,in_sigma,in_mutpb,in_tau):
    pop = [individual(0,individual_size) for i in range(pop_size)]
    for i in range(len(pop)):
        pop[i].genes = np.random.choice(final_results['solution'],1)[0]
        if np.random.uniform() < in_mutpb:
            pop[i] = mutGaussianAdaptive(pop[i],in_sigma,in_tau,1)
    return pop

In [63]:
def genetic_algorithm(train_enemies,in_sigma,in_tau,in_mutpb,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env,final_results

    train_enemies = np.array(train_enemies)
    pop = best_initializer(pop_size,in_sigma,in_mutpb,in_tau)
    hof_gain = {'best_solution':-np.inf,'max_value':-np.inf}
    hof_defeat = {'best_solution':-np.inf,'max_value':-np.inf}
    logbook = log_book()

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        pop = list(pool.map(play,pop))

        # record best solution
        for ind in pop:
            if ind.sum_gain>hof_gain['max_value']:
                hof_gain['best_solution'] = ind
                hof_gain['max_value'] = ind.sum_gain
            if ind.defeated>hof_defeat['max_value']:
                hof_defeat['best_solution'] = ind
                hof_defeat['max_value'] = ind.defeated
        
        logbook.create_log(pop,0,len(pop),train_enemies,hof_gain,hof_defeat)
        print("gen: 0")

        for g in range(ngen):
            # select parents
            parents = selTournament(individuals=pop,k=n_offspring*pop_size,tourn_size=tournsize_parent,train_enemies=train_enemies)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)        

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.evaluated]
            offspring = list(pool.map(play,offspring))

            # record best solution
            for ind in offspring:
                if ind.sum_gain>hof_gain['max_value']:
                    hof_gain['best_solution'] = ind
                    hof_gain['max_value'] = ind.sum_gain
                if ind.defeated>hof_defeat['max_value']:
                    hof_defeat['best_solution'] = ind
                    hof_defeat['max_value'] = ind.defeated

            # select the next generation of the population
            pop = selTournament(individuals=offspring,k=pop_size,tourn_size=tournsize_survival,train_enemies=train_enemies)

            logbook.create_log(pop,g+1,len(invalid_ind),train_enemies,hof_gain,hof_defeat)
            print("gen: "+str(g+1))
            
        pool.close()
        pool.join()
        
    display(logbook.pd)
    return hof_gain,hof_defeat,logbook
    

In [64]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [71]:
def objective(trial):
    
    params = {'train_enemies':trial.suggest_categorical("train_enemies",[[7,8],[2,5,6],[1,2,6,7]]),
              'in_sigma':trial.suggest_float("in_sigma", 0.1, 1., step=0.1),
              'in_tau':trial.suggest_float("in_tau", 0.1, 1., step=0.1),
              'in_mutpb':trial.suggest_float("in_mutpb", 0.1, 1., step=0.1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=50),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning_gain,hof_tuning_defeat
    iteration_logs = []
    for i in range(1):
        hof_gain,hof_defeat,logs = genetic_algorithm(**params)
        logs.pd['iteration'] = i+1
        iteration_logs.append(logs.pd)
        hof_tuning_gain.append(hof_gain)
        hof_tuning_defeat.append(hof_defeat)
    
    return pd.concat(iteration_logs).groupby('iteration')['hof_gain_max'].max()
    


In [72]:
import pickle
with open('/kaggle/input/tune-res-generalist/final_solutions_v2.pickle', 'rb') as handle:
    final_results = pickle.load(handle)

In [73]:
hof_tuning_gain,hof_tuning_defeat = [], []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=500)

[I 2023-10-15 15:45:02,969] A new study created in memory with name: GA
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [7, 8] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 5, 6] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 6, 7] which is of type list.
  warnings.warn(message)


gen: 0
gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
gen: 25
gen: 26
gen: 27
gen: 28
gen: 29
gen: 30


,gen,nevals,train_enemies_gain_mean,train_enemies_gain_max,train_enemies_gain_std,all_enemies_gain_mean,all_enemies_gain_max,all_enemies_defeated_max,hof_gain_max,hof_defeat_max
0,0,50,-57.092,114.8,116.196091,-312.716,249.0,5,249.0,5
1,1,244,39.396,148.8,72.364638,-241.748,-63.6,4,249.0,5
2,2,236,74.092,147.4,29.964171,-187.472,-15.8,4,249.0,5
3,3,244,75.492,147.6,20.911440,-185.560,-15.8,4,249.0,5
4,4,238,80.280,119.4,19.610038,-169.584,1.0,4,249.0,5
5,5,245,79.852,141.4,22.728337,-178.996,-55.6,4,249.0,5
6,6,239,84.556,124.6,22.251878,-166.872,-54.8,4,249.0,5
7,7,239,87.108,121.0,20.392595,-150.832,-22.6,4,249.0,5
8,8,237,87.432,122.6,21.045716,-142.200,-22.6,4,249.0,5
9,9,238,81.732,124.8,17.891388,-164.380,75.0,4,249.0,5


[I 2023-10-15 16:01:02,270] Trial 0 finished with value: 90.28000000000084 and parameters: {'train_enemies': [2, 5, 6], 'in_sigma': 1.0, 'in_tau': 0.9, 'in_mutpb': 0.9, 'pop_size': 50, 'n_offspring': 5, 'tournsize_parent': 1, 'cxpb': 0.8, 'cx_type': 'whole_aritmetic', 'alpha': 0.8, 'mutpb': 0.8, 'mut_type': 'adaptive', 'sigma': 2.5, 'indpb': 'random', 'tau': 0.9, 'tournsize_survival': 5, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 90.28000000000084.
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [7, 8] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 5, 6] which is of type list.
  warnings.warn(message)
/opt

gen: 0
gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
gen: 25
gen: 26
gen: 27
gen: 28
gen: 29
gen: 30


...

,gen,nevals,train_enemies_gain_mean,train_enemies_gain_max,train_enemies_gain_std,all_enemies_gain_mean,all_enemies_gain_max,all_enemies_defeated_max,hof_gain_max,hof_defeat_max
0,0,50,-3.204,187.2,1.101565e+02,22.592,249.0,6,249.0,6
1,1,48,175.436,187.2,2.318650e+01,187.392,249.0,5,249.0,6
2,2,53,187.200,187.2,2.842171e-14,199.200,199.2,5,249.0,6
3,3,47,187.200,187.2,2.842171e-14,199.200,199.2,5,249.0,6
4,4,43,187.200,187.2,2.842171e-14,199.200,199.2,5,249.0,6
5,5,60,187.200,187.2,2.842171e-14,199.200,199.2,5,249.0,6
6,6,51,187.200,187.2,2.842171e-14,199.200,199.2,5,249.0,6
7,7,40,187.200,187.2,2.842171e-14,199.200,199.2,5,249.0,6
8,8,49,187.200,187.2,2.842171e-14,199.200,199.2,5,249.0,6
9,9,44,187.200,187.2,2.842171e-14,199.200,199.2,5,249.0,6


[I 2023-10-15 19:02:15,942] Trial 24 finished with value: 224.1000000000007 and parameters: {'train_enemies': [1, 2, 6, 7], 'in_sigma': 0.2, 'in_tau': 0.5, 'in_mutpb': 0.6, 'pop_size': 50, 'n_offspring': 3, 'tournsize_parent': 3, 'cxpb': 0.2, 'cx_type': 'whole_aritmetic', 'alpha': 0.30000000000000004, 'mutpb': 0.2, 'mut_type': 'adaptive', 'sigma': 1.5, 'indpb': 0.2, 'tau': 0.6, 'tournsize_survival': 10, 'ngen': 30, 'verbose': 1}. Best is trial 10 with value: 224.1000000000007.
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [7, 8] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 5, 6] which is of type list.
  warnings.w

gen: 0
gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
gen: 25
gen: 26


Process ForkPoolWorker-123:
Process ForkPoolWorker-122:
Process ForkPoolWorker-124:
Process ForkPoolWorker-121:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self

KeyboardInterrupt: 

In [ ]:
#Trial 16 finished with value: 33.61333333333402 and parameters: {'train_enemies': [2, 5, 6], 'initial_range': 1, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 8, 'cxpb': 0.5, 'cx_type': 'whole_aritmetic', 'alpha': 0.2, 'mutpb': 1.0, 'mut_type': 'normal', 'sigma': 2.5, 'indpb': 0.6, 'tau': 0.30000000000000004, 'tournsize_survival': 10, 'ngen': 30, 'verbose': 1}

In [ ]:
#Trial 2 finished with value: 220.0800000000006 and parameters: {'train_enemies': [1, 2, 6, 7], 'in_sigma': 0.30000000000000004, 'in_tau': 0.6, 'in_mutpb': 0.2, 'pop_size': 50, 'n_offspring': 5, 'tournsize_parent': 8, 'cxpb': 0.30000000000000004, 'cx_type': 'whole_aritmetic', 'alpha': 0.6, 'mutpb': 0.7000000000000001, 'mut_type': 'normal', 'sigma': 0.5, 'indpb': 'random', 'tau': 1.0, 'tournsize_survival': 9, 'ngen': 30, 'verbose': 1}

In [13]:
params = {'train_enemies': [2, 5, 6],
          'initial_range': 1,
          'pop_size': 50,
          'n_offspring': 4,
          'tournsize_parent': 8,
          'cxpb': 0.5,
          'cx_type': 'whole_aritmetic',
          'alpha': 0.2,
          'mutpb': 1.0,
          'mut_type': 'normal',
          'sigma': 2.5,
          'indpb': 0.6,
          'tau': 0.3,
          'tournsize_survival': 10,
          'ngen': 500,
          'verbose': 1}

In [ ]:
def final_res(sol):
    global env
    res = []
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res.append(env.play(sol))
    return res

In [ ]:
"""
params = {'train_enemies':[7,8],
          'initial_range':1,
          'pop_size':50,
          'n_offspring':2,
          'tournsize_parent':5,
          'cxpb':0.7,
          'cx_type':'blend',
          'alpha':'random',
          'mutpb':0.3,
          'mut_type':'adaptive',
          'sigma':2,
          'indpb':0.8,
          'tau':0.3,
          'tournsize_survival':10,
          'ngen':10,
          'verbose':1}
          
""""